### Tests for Applying Gains Matrix

In [3]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))

In [5]:
import cupy as cp
import numpy as np
from utils import *
from zp_puregpu_funcs_py import *
from invcov import *

In [6]:
apply_gains_to_mat?
# undo_zeropad?

In [7]:
summarize_benchmark_results?

In [8]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 4
n_bl = 30
n_gains = 2*n_bl
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_bl//2)
ant_2_array = cp.arange(n_bl//2)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

The edges of the redundant blocks have indices[ 0  8 10 14 24 30]


In [9]:
applied_gains = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)

In [10]:
print(f"Input mat has shape {sim_diff_mat_3d.shape}")
print()
print(f"Gains applied mat has shape {applied_gains.shape}")
print()
print(f"The mat with gains applied looks like:\n\n{applied_gains}")

Input mat has shape (5, 10, 3)

Gains applied mat has shape (5, 10, 3)

The mat with gains applied looks like:

[[[1.79832601e-01 2.37720325e-01 1.90664589e-02]
  [1.02341087e-01 1.22167161e-01 8.87506807e-02]
  [1.09012626e+00 1.06705123e+00 1.44143714e+00]
  [2.26990831e-01 2.73063914e-01 4.44323651e-01]
  [1.91781932e-02 4.82757735e-01 3.59954669e-01]
  [1.04729585e-01 1.09782409e-03 2.48338803e-02]
  [3.42354179e-03 3.59718579e-05 5.67986568e-04]
  [4.51581109e-03 5.09967856e-03 4.97524668e-04]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[2.50738809e-01 2.41775457e-02 7.09466708e-02]
  [6.28242280e-02 1.31826499e-01 3.28829449e-01]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]
  [0

#### Random Debugging tests

In [11]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

(2, 5, 1)
(2, 5, 3)
(10,)
[[[1.86114327e-03 1.56023192e-04 5.59169741e-01]
  [4.52870338e-01 3.03159589e-02 2.80962128e-01]
  [2.82347109e-02 2.49786608e-01 2.39384022e-01]
  [2.83912693e-01 6.00083282e-01 9.29675871e-01]
  [9.63820261e-01 2.64099350e-01 6.48553453e-02]]

 [[1.69096792e-01 6.76890205e-01 1.42107647e-01]
  [2.01493494e-01 4.47112160e-01 6.92293322e-01]
  [3.09090014e-01 6.84009080e-01 9.99210709e-01]
  [2.30845735e-01 3.81976866e-01 2.80194881e-01]
  [4.47762883e-01 4.97842489e-01 4.97491916e-01]]]
[[[1.86114327e-03 1.56023192e-04 5.59169741e-01]
  [2.82347109e-02 2.49786608e-01 2.39384022e-01]
  [9.63820261e-01 2.64099350e-01 6.48553453e-02]]

 [[1.69096792e-01 6.76890205e-01 1.42107647e-01]
  [3.09090014e-01 6.84009080e-01 9.99210709e-01]
  [4.47762883e-01 4.97842489e-01 4.97491916e-01]]]


In [12]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])

[0.27548071 0.20219736 0.02630517 0.02697913 0.68606275 0.36724726
 0.36389394 0.47875717 0.72525188 0.2169671 ]
[0.27548071 0.20219736 0.02630517 0.02697913 0.68606275]

[0.36724726 0.36389394 0.47875717 0.72525188 0.2169671 ]

[0.27548071 0.20219736 0.02630517 0.02697913 0.68606275 0.36724726
 0.36389394 0.47875717 0.72525188 0.2169671 ]

[0.27548071 0.20219736 0.02630517 0.02697913 0.68606275]
[0.36724726 0.36389394 0.47875717 0.72525188 0.2169671 ]


#### Check ZP computes the same thing as corrcal

In [13]:
#Current corrcal gain application function
def apply_gains_to_mat_corrcal(gains, mat, ant_1_array, ant_2_array):
    """Apply a gain-like matrix to a provided matrix."""
    complex_gains = gains[::2] + 1j*gains[1::2]
    gain_mat = (
        complex_gains[ant_1_array,None] * complex_gains[ant_2_array,None].conj()
    )
    out = np.zeros_like(mat)
    out[::2] = gain_mat.real * mat[::2] - gain_mat.imag * mat[1::2]
    out[1::2] = gain_mat.imag * mat[::2] + gain_mat.real * mat[1::2]
    return out

In [14]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 15
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)


The edges of the redundant blocks have indices[ 0 16 18 20 30]


In [15]:
out_corrcal = apply_gains_to_mat_corrcal(sim_gains, sim_diff_mat, ant_1_array, ant_2_array)
out_gpu = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, cp)
out_gpu_corrcal = apply_gains_to_mat(sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, cp, False)
out_gpu_resh = undo_zeropad(out_gpu, edges, cp)

In [16]:
#should return True if my routine computes the same thing as the current corrcal routine
print(cp.allclose(out_gpu_resh, out_gpu_corrcal))
print(cp.allclose(out_gpu_resh, out_corrcal))

True
True


#### Timing test for np vs cp versions

In [17]:
#simulation params relevant for testing application of gains to a matrix
n_bl = 240000  # something like 2*N_ant^2 for Re/Im split
n_ant = 512
n_gains = 2*n_bl  #one for each antenna and one for each Re and Im part
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
if xp == cp:

    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
    sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant

    #leave as n_bl/2 since indexex into cplex gains which are not Re/Im split
    ant_1_array = cp.arange(n_bl//2) 
    ant_2_array = cp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

elif xp == np:
    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig).astype('float64')
    sim_gains = xp.random.rand(n_gains).astype('float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
    ant_1_array = xp.arange(n_bl//2)
    ant_2_array = xp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    # zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    # sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

print(len(edges))

The edges of the redundant blocks have indices[     0    640    770    930   1162   1616   1742   4276   5208   5776
   6002   6608   6792   7380   7454   7892   7900   8054   8148   8178
   8284   8436   9212  10036  10812  10884  10904  11402  11426  12078
  12502  12586  12802  13148  13940  14096  15918  16620  17010  17264
  17506  17514  17614  19040  19470  20150  20464  20990  21912  22088
  22222  22462  24084  24310  24466  24990  26270  27170  27224  27728
  28044  28972  29298  30254  30284  30756  30884  31396  32250  32590
  32828  35958  36230  36712  36916  37140  37188  37218  37926  38364
  38758  38920  39418  40312  40488  40740  41172  41772  42538  42592
  43100  43622  43920  44208  44334  44618  44872  45092  45328  45472
  45780  46030  46070  46208  46470  46664  46700  47256  47264  47286
  48022  48418  48650  50006  50344  50814  51226  51374  51924  52140
  52738  53220  53338  54450  54960  54974  55666  55684  55714  55848
  56142  56238  56250  56270  5

In [18]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, xp, False)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.001015s
Time on gpu: 0.001206s


In [19]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, xp, True)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.002877s
Time on gpu: 0.004999s


### Random debugging....

In [5]:
import numpy as np
mat = np.random.rand(3, 2, 3)
print(mat)
gm = np.arange(6)
gm = gm.reshape(3, 2, 1)
print(gm)
print()
print(gm * mat)

[[[0.25805107 0.10975667 0.0127483 ]
  [0.60295261 0.68578718 0.30930862]]

 [[0.26352463 0.32566188 0.47140867]
  [0.96412682 0.77976336 0.89265702]]

 [[0.90555489 0.9337359  0.94056508]
  [0.29599784 0.34932493 0.94214839]]]
[[[0]
  [1]]

 [[2]
  [3]]

 [[4]
  [5]]]

[[[0.         0.         0.        ]
  [0.60295261 0.68578718 0.30930862]]

 [[0.52704926 0.65132376 0.94281734]
  [2.89238046 2.33929009 2.67797105]]

 [[3.62221958 3.73494359 3.7622603 ]
  [1.47998921 1.74662463 4.71074195]]]


In [8]:
a = np.random.rand(4,4)
print(a)
print(a[::2])

[[0.41763539 0.56619337 0.25594049 0.47654163]
 [0.75982616 0.21497384 0.31961329 0.37099003]
 [0.47325748 0.2578755  0.3074834  0.40192564]
 [0.93266381 0.53751097 0.96622743 0.17613385]]
[[0.41763539 0.56619337 0.25594049 0.47654163]
 [0.47325748 0.2578755  0.3074834  0.40192564]]


In [11]:
a = np.arange(15)
b = np.arange(15)

print(a[:, None]* b[:, None])

[[  0]
 [  1]
 [  4]
 [  9]
 [ 16]
 [ 25]
 [ 36]
 [ 49]
 [ 64]
 [ 81]
 [100]
 [121]
 [144]
 [169]
 [196]]
